In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
workspace = '/content/drive/MyDrive/Courses/CS410/MP4/'

In [ ]:
import os
os.path.exists(workspace)

True

# Evaluating Search Sytems using TREC EVAL


This contains an example on how to go ahead and evaluate models using ir-measures, which is a wrapper on the popular TRECEVAL framework. TRECEVAL is the common framework for format for evaluating search sysems

In [ ]:
%%capture
!pip install ir_measures

In [ ]:
import ir_measures
from ir_measures import *

In [ ]:
def load_trec(filename):
    qid2docid = {}
    with open(filename, 'r') as f:
        for l in f:
            l = l.strip().split('\t')
            qid = int(l[0])
            doc_id = int(l[2])
            rank = int(l[3])
            score = float(l[4])
            if qid not in qid2docid:
                qid2docid[qid] = {}
            qid2docid[qid][rank] = (doc_id, score)
    return qid2docid

The example shown below will load the ranking, write it out in TREC format and then evaluate them on the previously generate relevance labels

In [ ]:
bm25_candidates = load_trec(workspace + 'data/bm25-top1000.trec')

In [ ]:
bm25_candidates[1][1] # First value is the doc_id and the second is score.

(1936, 6.0995)

In [ ]:
len(bm25_candidates)

2232

In [ ]:
# i = 0
# top_n = 10 # keep top 10 in ranking

# with open(workspace + 'ranking_candidate.trec', 'w') as w:
#     for i in range(1, len(bm25_candidates)): #query_id starts at 1
#         i += 1 
#         if i in bm25_candidates:
#             candidates = bm25_candidates[i]
#             for j in range(1, min(len(candidates),(top_n+1))): #rank starts at 1
#                 w.write("{}\t0\t{}\t{}\t{}\tcandidate-run\n".format(i, candidates[j][0], j, candidates[j][1]))  #QUERY_ID\t0\tDOCUMENT_ID\tRANK\tSCORE\trun_id

In [ ]:
top_n = 10 # keep top 10 in ranking

with open(workspace + 'ranking_candidate.trec', 'w') as w:
    for i in range(1, len(bm25_candidates)+1): #query_id starts at 1 
        if i in bm25_candidates:
            candidates = bm25_candidates[i]
            for j in range(1, min(len(candidates),top_n)+1): #rank starts at 1
                w.write("{}\t0\t{}\t{}\t{}\tcandidate-run\n".format(i, candidates[j][0], j, candidates[j][1]))  #QUERY_ID\t0\tDOCUMENT_ID\tRANK\tSCORE\trun_id

In [ ]:
# candidates

In [ ]:
"{}\t0\t{}\t{}\t{}\tcandidate-run\n".format(i, candidates[j][0], j, candidates[j][1])  #QUERY_ID\t0\tDOCUMENT_ID\tRANK\tSCORE\trun_id

'2232\t0\t1717\t10\t3.8778\tcandidate-run\n'

In [ ]:
qrels = ir_measures.read_trec_qrels(workspace + 'data/qrels.trec')
run = ir_measures.read_trec_run(workspace + 'ranking_candidate.trec')

In [ ]:
# print( ir_measures.calc_aggregate([NDCG@5,NDCG@10,NDCG@3,NDCG@20,R@10,R@100,R@1000,], qrels, run) )

In [ ]:
names = [NDCG@3, NDCG@5, NDCG@10, NDCG@20, R@5, R@10, R@100, R@1000]
evals = ir_measures.calc_aggregate(names, qrels, run)

In [ ]:
for name in names:
  print("{:10s} = {:.3f}".format(str(name), evals[name]))

nDCG@3     = 0.516
nDCG@5     = 0.494
nDCG@10    = 0.472
nDCG@20    = 0.408
R@5        = 0.235
R@10       = 0.365
R@100      = 0.365
R@1000     = 0.365
